In [ ]:
import seaborn as sns
df = sns.load_dataset('tips')


In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
# Lets Take 'Time' as Output feature 
df['time'].unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df.head()

In [ ]:
df['time'] = encoder.fit_transform(df['time'])

In [ ]:
df.time.unique()

In [ ]:
## changing in dependent and independent feature 
X = df.drop(labels=['time'], axis=1 )
y= df['time']

In [ ]:
# X , y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y, random_state=42 ,test_size=0.2)

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


In [ ]:
# [feature for feature in df.columns
#  if df[feature].dtype!='category']

In [ ]:
# division of numerical and catogrical feature --seggregation
categorical_cols = ['sex' , 'smoker', 'day']
numerical_cols = ['total_bill', 'tip', 'size']

In [ ]:
# Numerical and categorical  pipelining for feature engineering
num_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')) , #missing values handled
        ('scaler', StandardScaler()) # scaling handled # feature scaling 
    ]
)

cat_pipeline = Pipeline(
    steps=[
         ('imputer', SimpleImputer(strategy='most_frequent')) , 
         ('onehotencoder', OneHotEncoder()) # vategorical features to numerical 
    ]
)

In [ ]:
#  combine those two pipes 
preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline , numerical_cols) , 
    ('cat_pipeline', cat_pipeline, categorical_cols)
])

In [ ]:
X_train =  preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# model training ko automate karenge ab
from sklearn.linear_model import LogisticRegression  
from sklearn.tree import DecisionTreeClassifier
models = {
    'Random Forest' : RandomForestClassifier() , 
    'Logistic Regression' : LogisticRegression() ,
    'Decision Tree' : DecisionTreeClassifier() ,
}

In [ ]:
from sklearn.metrics import accuracy_score 
  

In [ ]:
def evaluate_model(X_train,y_train,X_test,y_test,models):
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        # Train model 
        model.fit(X_train, y_train)

        # predict testing data 
        y_pred = model.predict(X_test)
        
        # get Accuracy 
        accuracy = accuracy_score(y_pred , y_test)

        report[list(models.keys())[i]] = accuracy

    return report


In [ ]:
scores = evaluate_model(X_train, y_train, X_test, y_test , models)
scores

In [ ]:
# max(sorted(scores.values()))

In [ ]:
classifier  = RandomForestClassifier()
params = {'max_depth': [3,5,10,None] , 
          'n_estimators' : [100,200,300 ] , 
          'criterion' :['gini','entropy']
          } 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
cv = RandomizedSearchCV(classifier  , param_distributions=params , scoring='accuracy' , cv=5)

In [ ]:
cv.fit(X_train,y_train)

In [ ]:
cv.best_params_